
## 1. 지난시간 복습

Cross-entrophy loss는 다음과 같이 정의한다. 

$CELoss = -  \sum^C_{i=1} (y_i \cdot \log \hat{y_i})$


여기서 C는 분류할 클래스의 개수이다. 
우리는 Loss를 모든 데이터에 기반하여 계산할 것이므로, 최종으로 구할 L는 모든 데이터 M개에 관한 CELoss의 평균을 이용한다. 

$L = - \sum^{M}_{j=1} \sum^C_{i=1} (y_i^j \cdot \log \hat{y_i^j})$

일반적으로 CELoss는 Softmax가 적용된 출력을 입력으로 받으며, softmax+CELoss를 입력값으로 미분하게 될 때 매우 간결한 형태의 analytic gradient를 얻는다. 

- $\frac{\partial L}{\partial z_i} = p_i - y_i$


- 여기서 $z_i$는 softmax에 들어가는 입력 (혹은 앞 레이어의 출력)



CELoss + Softmax 결합 모듈의 미분 증명 

- https://levelup.gitconnected.com/killer-combo-softmax-and-cross-entropy-5907442f60ba


In [2]:
import numpy as np

# softmax function: np.exp에 너무 큰값이 들어가지 않도록 수정
def Softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# CELoss 
def CELoss(Y_hat, labels): 
    n_data = len(labels)
    loss = 0 
    for i in range(n_data):        
        celoss = np.sum(- (labels[i]*np.log(Y_hat[i])))
        loss += celoss/n_data
    return loss
    
# 데이터
x_1 = np.array([[56, 231],
                   [24, 1]])
x_2 = np.array([[120, 30],
                   [24, 0]])
x_3 = np.array([[20, 31],
                   [240, 100]])
x_4 = np.array([[56, 201],
                   [22, -10]])
x_5 = np.array([[140, 27],
                   [30, 10]])
x_6 = np.array([[25, 30],
                   [230, 110]])


# one-hot vector 정의하기
y_1 = np.array([1,0,0])   #label_cat
y_2 = np.array([0,1,0])   #label_dog
y_3 = np.array([0,0,1])   #label_ship 
y_4 = np.array([1,0,0])   #label_cat_2
y_5 = np.array([0,1,0])   #label_dog_2
y_6 = np.array([0,0,1])   #label_ship_2 


#  데이터셋
X = np.array([x_1, x_2, x_3, x_4, x_5, x_6]) # data
Y = np.array([y_1, y_2, y_3, y_4, y_5, y_6]) # label

print(X.shape)
print(Y.shape)

(6, 2, 2)
(6, 3)


## 실습과제 . 지난시간에 구현한 뉴럴넷 Class 형태로 재구현

지난 시간에 구현하였던 간단한 선형분류기+Softmax+CELoss 구조를 갖는 뉴럴네트워크를 아래와 같이 python class로 구현하였다. 

- 구조: x -> Linear(4,3) -> Softmax -> CELoss <- y_hat

여기서 Linear(in, out)은 입력in개와 출력 out개를 갖는 선형분류기를 뜻한다. Backward 함수를 구현하여 본 뉴럴넷의 학습을 동작하게 만드시오. 각 레이어에 대한 local gradient 계산법은 아래에 첨부하였다. 

------------------------------

각 레이어에 대한 미분값

Derivative of CELoss with softmax :

- $\frac{\partial L}{\partial x_i} = p_i - y_i$

- 여기서 $x_i$는 softmax에 들어가는 입력 (혹은 앞 레이어의 출력 $z_i$)

- ($L = CELoss(softmax(x), y) = CELoss(p,y)$) 

- https://levelup.gitconnected.com/killer-combo-softmax-and-cross-entropy-5907442f60ba

Derivative of Linear layer (Y = WX):

- $\frac{\partial L}{\partial W} = \frac{\partial L}{\partial Y} X^T$

- $\frac{\partial L}{\partial X} = W^T \frac{\partial L}{\partial Y} $


- http://cs231n.stanford.edu/handouts/linear-backprop.pdf





In [3]:
# Single-layer Neural Network class

class NN_1layer():
    def __init__(self, **kwargs):
        self.W_l1 = np.random.rand(3,4)
        self.activation = Softmax
        self.Loss = CELoss
        self.learning_rate = 0.0001

    def Forward(self, X):
        # initial prediction
        Y_hat = []
        
        for x in X: 
            # layer 1
            x = np.matmul(self.W_l1, x.flatten())
          
            # softmax
            p = self.activation(x)

            Y_hat.append(p)
        return Y_hat
        
    def getLoss(self, X, Y):
        # y_hat -> CELoss <- y
        Y_hat = self.Forward(X)
        return self.Loss(Y_hat, Y)

    def Backward(self, Y_hat, X, Y): 
        dW1Loss = np.zeros_like(self.W_l1)
        N = X.shape[0] # 데이터 개수

        ####################################
        ##### 이곳에 코드를 작성하시오 #####
        # backpropagation 구현 힌트 
        # 1. softmax와 CELoss는 한개의 레이어로 간주하고 미분
        #   x -> [Linear(4,3)] -> [Softmax -> CELoss] <- y_hat
        # 2. 두 레이어의 gradient 공식은 위의 각 레이어 미분 공식을 분석
        #   분석 시 함수, 입력, 출력 기호를 명확하게 파악할 것!
        
        # Backpropagation 공식
        #   Downstream gradient = UpstreamGradient * LocalGradient
        # - 각 레이어 별 LocalGradient 계산법을 파악
        # - 각 레이어 별 UpstreamGradient가 무엇인지 파악
        # - 위 두가지를 곱하여 downstreamGradient를 계산
      
        ##### 참고 사항. np 행렬의 형상 차이에 주의 
        # shape (N,)은 1차원 배열로써 행렬의 transpose가 적용되지 않으므로 
        # reshape(N,1)로 차원을 명시한 뒤에 transpose를 적용하시오. 
        # 사용 예)  x.reshape(4,1).T

        # 개별 데이터 단위 연산
        for i, x in enumerate(X): 
            dCELoss = Y_hat[i] - Y[i]
            dW1Loss += np.matmul(dCELoss.reshape(3,1), x.reshape(4,1).T)


        # 데이터 행렬 

        ####################################

        return dW1Loss/N

    def training(self, X, Y, n_epoch=100):
        iterations = 0
        while iterations < n_epoch: 
            # gradient descent algorithm 
            Y_hat = self.Forward(X)
            dWLoss = self.Backward(Y_hat, X, Y)

            self.W_l1 -= dWLoss*self.learning_rate
            print(f'iteration: {iterations+1}, loss: {self.getLoss(X,Y)}')
            iterations += 1

my_NeuralNet = NN_1layer()
my_NeuralNet.training(X, Y)

# 학습이 잘되었다면 [1,0,0] 과 같은 형태로 출력
print(my_NeuralNet.Forward(X))

# 학습이 잘되었다면 매우 작은값(예: 0.01) 형태가 출력된다.
print(my_NeuralNet.getLoss(X, Y))


iteration: 1, loss: 53.32741088981144
iteration: 2, loss: 51.27258995044743
iteration: 3, loss: 49.21776945234153
iteration: 4, loss: 47.16294934510085
iteration: 5, loss: 45.10812958455894
iteration: 6, loss: 43.05331013196445
iteration: 7, loss: 40.99849095327901
iteration: 8, loss: 38.94367201856942
iteration: 9, loss: 36.88885330148117
iteration: 10, loss: 34.83403477878206
iteration: 11, loss: 32.77921642996658
iteration: 12, loss: 30.724398236912602
iteration: 13, loss: 28.66958018358326
iteration: 14, loss: 26.614762255768092
iteration: 15, loss: 24.55994444085801
iteration: 16, loss: 22.50512672765023
iteration: 17, loss: 20.450309106185454
iteration: 18, loss: 18.395491567843223
iteration: 19, loss: 16.34067411452207
iteration: 20, loss: 14.28585712131781
iteration: 21, loss: 12.231055512249021
iteration: 22, loss: 10.17685274875087
iteration: 23, loss: 8.145186969063381
iteration: 24, loss: 6.5042429804285575
iteration: 25, loss: 5.419243822360853
iteration: 26, loss: 4.35383

-----------------------------------------

# 심화 연습문제 (optional)

위 네트워크 구조를 아래와 같이 2 layer로 바꾸어 보고, 학습을 성공시켜 보시오. init(), Forward와 Backward 함수를 수정해야 한다. 

- 구조: x -> Linear(4,4) -> ReLU ->  Linear(4,3) -> Softmax -> CELoss <- y_hat

ReLU는 아래의 구현을 참고하시오. 




In [4]:
# ReLU
def ReLU(x):
    return np.maximum(0, x)

# derivatives of ReLU
def dReLU(x):
    # if (x > 0):
    #     return 1
    # if (x <= 0):
    #     return 0
    return 1 * (x > 0)


In [21]:
# Two-layer Neural Network class
import numpy as np

class NN_2layer():
    def __init__(self, **kwargs):
        self.W_l1 = np.random.rand(4,4)
        self.W_l2 = np.random.rand(3,4)
        self.activation = Softmax
        self.Loss = CELoss
        self.learning_rate = 0.001
        self.z_2 = 0

    def Forward(self, X):
        # initial prediction
        Y_hat = []
        
        for x in X: 
            # layer 1
            z_1 = np.matmul(self.W_l1, x.flatten())
          
            # layer 2
            z_2 = ReLU(z_1)
            self.z_2 = z_2

            # layer 3
            z_3 = np.matmul(self.W_l2, z_2)
                        
            # softmax
            p = self.activation(z_3)

            Y_hat.append(p)
        return Y_hat
        
    def getLoss(self, X, Y):
        # y_hat -> CELoss <- y
        Y_hat = self.Forward(X)
        return self.Loss(Y_hat, Y)

    def Backward(self, Y_hat, X, Y): 
        dW1Loss = np.zeros_like(self.W_l1)
        dW2Loss = np.zeros_like(self.W_l2)
        N = X.shape[0] # 데이터 개수

        ####################################
        ##### 이곳에 코드를 작성하시오 #####
        # backpropagation 구현 힌트 
        # 1. softmax와 CELoss는 한개의 레이어로 간주하고 미분
        #   x -> [Linear(4,3)] -> [Softmax -> CELoss] <- y_hat
        # 2. 두 레이어의 gradient 공식은 위의 각 레이어 미분 공식을 분석
        #   분석 시 함수, 입력, 출력 기호를 명확하게 파악할 것!
        
        # Backpropagation 공식
        #   Downstream gradient = UpstreamGradient * LocalGradient
        # - 각 레이어 별 LocalGradient 계산법을 파악
        # - 각 레이어 별 UpstreamGradient가 무엇인지 파악
        # - 위 두가지를 곱하여 downstreamGradient를 계산
      
        ##### 참고 사항. np 행렬의 형상 차이에 주의 
        # shape (N,)은 1차원 배열로써 행렬의 transpose가 적용되지 않으므로 
        # reshape(N,1)로 차원을 명시한 뒤에 transpose를 적용하시오. 
        # 사용 예)  x.reshape(4,1).T

        for i, x in enumerate(X): 
            # CElayer Loss
            dz3Loss = Y_hat[i] - Y[i]

            # dW2Loss
            dW2Loss += np.matmul(dz3Loss.reshape(3,1), self.z_2.reshape(4,1).T)

            # dz2Loss
            dz2Loss = np.matmul(self.W_l2.T , dz3Loss.reshape(3,1))

            # dz2Loss
            dz1Loss = dReLU(dz2Loss) * dz2Loss

            # dW1Loss
            dW1Loss += np.matmul(dz1Loss.reshape(4,1), x.reshape(4,1).T)

        ####################################

        return dW1Loss/N, dW2Loss/N

    def training(self, X, Y, n_epoch=100):
        iterations = 0
        while iterations < n_epoch: 
            # gradient descent algorithm 
            Y_hat = self.Forward(X)
            dW1Loss, dW2Loss = self.Backward(Y_hat, X, Y)

            self.W_l1 -= dW1Loss*self.learning_rate
            self.W_l2 -= dW2Loss*self.learning_rate
            print(f'iteration: {iterations+1}, loss: {self.getLoss(X,Y)}')
            iterations += 1

my_NeuralNet = NN_2layer()
my_NeuralNet.training(X, Y)

# 학습이 잘되었다면 [1,0,0] 과 같은 형태로 출력
print(my_NeuralNet.Forward(X))

# 학습이 잘되었다면 매우 작은값(예: 0.01) 형태가 출력된다.
print(my_NeuralNet.getLoss(X, Y))


iteration: 1, loss: 137.89727250142252
iteration: 2, loss: 69.09063457346375
iteration: 3, loss: 34.372926513882746
iteration: 4, loss: 43.849807834622325
iteration: 5, loss: 57.130106036466586
iteration: 6, loss: 29.858920224417634
iteration: 7, loss: 36.15970833723573
iteration: 8, loss: 46.06527762294829
iteration: 9, loss: 25.661219152411178
iteration: 10, loss: 28.903151287994522
iteration: 11, loss: 35.842611571837935
iteration: 12, loss: 21.34826141224106
iteration: 13, loss: 22.12422630416762
iteration: 14, loss: 29.297385122671905
iteration: 15, loss: 0.04364930155451743
iteration: 16, loss: 0.0008920409560711436
iteration: 17, loss: 0.0008178390161729273
iteration: 18, loss: 0.0007553871811224882
iteration: 19, loss: 0.0007020651028355893
iteration: 20, loss: 0.0006559835415822302
iteration: 21, loss: 0.0006157446659152544
iteration: 22, loss: 0.0005802912241923469
iteration: 23, loss: 0.0005488082659554359
iteration: 24, loss: 0.0005206571622374424
iteration: 25, loss: 0.000